In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt


In [45]:
def extract_answer(text):
    try:
        text_processed = text.split('"answer":')[-1].split('\n')[0].strip().split('"')[1].strip()
    except:
        text_processed = text.split('"answer":')[-1].split('\n')[0].strip()
    return text_processed


In [46]:
PROMPT_RESPONSE_PATH = "../../../data/analysis_results/Llama_2_13b_chat_hf_prompt_based_two_hop_mcq_response.csv"

RAG_RESPONSE_PATH = "../../../data/analysis_results/Llama_2_13b_chat_hf_node_retrieval_rag_based_two_hop_mcq_response.csv"


In [47]:
# prompt_response_df = pd.read_csv(PROMPT_RESPONSE_PATH)
rag_response_df = pd.read_csv(RAG_RESPONSE_PATH)

# prompt_response_df.loc[:, 'extracted_answer'] = prompt_response_df['llm_answer'].apply(extract_answer)
rag_response_df.loc[:, 'extracted_answer'] = rag_response_df['llm_answer'].apply(extract_answer)


In [50]:
rag_response_df[rag_response_df.correct_answer == rag_response_df.extracted_answer].shape

(112, 4)

In [ ]:
def evaluate(df):
    correct = df[df.correct_answer == df.extracted_answer]
    incorrect = df[df.correct_answer != df.extracted_answer]
    correct_frac = correct.shape[0]/df.shape[0]
    incorrect_frac = incorrect.shape[0]/df.shape[0]
    return correct_frac, incorrect_frac

niter = 100
nsample = 30

prompt_correct_frac_list = []
rag_correct_frac_list = []
for _ in tqdm(range(niter)):
    prompt_response_df_sample = prompt_response_df.sample(n=nsample)
    prompt_correct_frac, prompt_incorrect_frac = evaluate(prompt_response_df_sample)

    rag_response_df_sample = rag_response_df.iloc[prompt_response_df_sample.index]
    rag_correct_frac, rag_incorrect_frac = evaluate(rag_response_df_sample)

    prompt_correct_frac_list.append(prompt_correct_frac)
    rag_correct_frac_list.append(rag_correct_frac)
